Palworlds

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd

url = 'https://palworld.wiki.gg/wiki/Pals'
response = requests.get(url)

###  pals_data.csv
####  colnames: 'No.', 'Name', 'Elements'

In [ ]:
names = []
numbers = []
elements = []
response = requests.get(url)
if response.status_code == 200:
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')
    tables = soup.find_all('table')
    for table in tables:
        tbody = table.find('tbody')
        if tbody:
            trs = tbody.find_all('tr')
            for tr in trs[1:]:
                name_a_tag = tr.find('span', class_='iconlink').find_all('a')[1]
                name = name_a_tag.get_text(strip=True)
                number = tr.find_all('td')[1].get_text(strip=True)
                ele = []
                for span_tag in tr.find_all('span'):
                    if span_tag.name == 'span' and "ele-" in ' '.join(span_tag.get("class", [])):
                        element_a_tag = span_tag.find('a')
                        element = element_a_tag.get_text(strip=True) if element_a_tag else "Unknown"
                        ele.append(element)
                element = ', '.join(ele) if ele else "Unknown"
                names.append(name)
                numbers.append(number)
                elements.append(element)
            df = pd.DataFrame({'No.': numbers,
                               'Name': names,
                               'Elements': elements})
            csv_file_path = 'pals_data.csv'
            df.to_csv(csv_file_path, index=False) 
            
                


### Images files
#### icon: name_icon.png
#### Screenshot: name.png

In [ ]:
def download_image(image_url, image_name, folder="images"):
    image_path = os.path.join(folder, image_name)
    response = requests.get(image_url)
    
    if response.status_code == 200:
        with open(image_path, 'wb') as f:
            f.write(response.content)
        print(f"download：{image_path}")
    else:
        print(f"error：{response.status_code}")

In [ ]:
def extract_image_urls(soup):
    image_urls = []

    screenshot_div = soup.find('div', id='pi-tab-1')
    if screenshot_div:
        screenshot_img = screenshot_div.find('img')
        if screenshot_img:
            image_urls.append(screenshot_img['src'])

    icon_div = soup.find('div', id='pi-tab-2')
    if icon_div:
        icon_img = icon_div.find('img')
        if icon_img:
            image_urls.append(icon_img['src'])

    return image_urls

In [ ]:
data=pd.read_csv('pals_data.csv')
# names = data['Name'].tolist()
index = data.index[data['Name'] == 'Petallia'].tolist()
names = data['Name'].iloc[index[0]:].tolist()
for name in names:
    name=name.replace(' ','_')
    url=f'https://palworld.wiki.gg/wiki/{name}'
    response = requests.get(url)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')

    image_urls = extract_image_urls(soup)
    for i, image_url in enumerate(image_urls):
        if 'icon' in image_url:
            image_name = f'{name}_icon.png'
        else:
            image_name = f'{name}.png'
        
        image_url=f'https://palworld.wiki.gg{image_url}'
        download_image(image_url, image_name)
 
            
    

download：images/Petallia.png
download：images/Petallia_icon.png
download：images/Reptyro.png
download：images/Reptyro_icon.png
download：images/Reptyro_Cryst.png
download：images/Reptyro_Cryst_icon.png
download：images/Kingpaca.png
download：images/Kingpaca_icon.png
download：images/Kingpaca_Cryst.png
download：images/Kingpaca_Cryst_icon.png
download：images/Mammorest.png
download：images/Mammorest_icon.png
download：images/Mammorest_Cryst.png
download：images/Mammorest_Cryst_icon.png
download：images/Wumpo.png
download：images/Wumpo_icon.png
download：images/Wumpo_Botan.png
download：images/Wumpo_Botan_icon.png
download：images/Warsect.png
download：images/Warsect_icon.png
download：images/Warsect_Terra.png
download：images/Warsect_Terra_icon.png
download：images/Fenglope.png
download：images/Fenglope_icon.png
download：images/Felbat.png
download：images/Felbat_icon.png
download：images/Quivern.png
download：images/Quivern_icon.png
download：images/Quivern_Botan.png
download：images/Quivern_Botan_icon.png
downloa

roco kingdom

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

import time
import pandas as pd
import os
import urllib.request

In [ ]:
url = 'https://rocokingdom.huijiwiki.com/wiki/%E5%AE%A0%E7%89%A9%E5%9B%BE%E9%89%B4'

def open_driver(url):
    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-logging']) # for ignore warning and error
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    return driver


###  roco_kingdom_data_year.txt
####  colnames: 'No.', 'Name', 'Elements'

In [ ]:
year=2014
i=year-2008

driver = open_driver(url)
per_year_button = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH,f'/html/body/div[2]/div/div[1]/main/article/section[1]/div/div/div/div[2]/div/div/div[{i}]'))
)
per_year_button.click()

WebDriverWait(driver, 30).until(
    EC.invisibility_of_element_located((By.XPATH, "//div[@class='pet-name' and text()='苦行骆驼']")))


time.sleep(10)
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "pet-card")))
# time.sleep(10)

page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')
pet_cards = soup.find_all(class_="pet-card")

names = []
numbers = []
elements = []
for pet_card in pet_cards:
    number = pet_card.find('div', class_='pet-title huiji-tt').get('data-name')
    name = pet_card.find('div', class_='pet-name').text
    element = [img.get('title') for img in pet_card.find_all('img')]
    element = element[1:len(element)-1]
    element = [x for x in element if x is not None]
    element = '，'.join(element)
    names.append(name)
    numbers.append(number)
    elements.append(element)

df = pd.DataFrame({'No.': numbers,
                    'Name': names,
                    'Elements': elements })
    
    # encoding problem
    # csv_file_path = 'roco_kingdom_data_2010.csv'
    # df.to_csv(csv_file_path, index=False, encoding='utf-8') 

txt_file_path = f'roco_kingdom_data_{year}.txt'
df.to_csv(txt_file_path, index=False, sep='\t', encoding='utf-8')



The chromedriver version (130.0.6723.58) detected in PATH at /usr/local/bin/chromedriver might not be compatible with the detected chrome version (131.0.6778.109); currently, chromedriver 131.0.6778.87 is recommended for chrome 131.*, so it is advised to delete the driver in PATH and retry


### Images files
#### images_year/No._name.png


In [ ]:
import urllib.request

In [ ]:
year=2014
i=year-2008

url = 'https://rocokingdom.huijiwiki.com/wiki/%E5%AE%A0%E7%89%A9%E5%9B%BE%E9%89%B4'

# Terminal: /Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome --remote-debugging-port=9222
def create_driver():
    chrome_options = Options()
    chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

driver = create_driver()
driver.get(url)
time.sleep(2)
opener = urllib.request.build_opener()
opener.addheaders = [('User-Agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')]
urllib.request.install_opener(opener)

per_year_button = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH,f'/html/body/div[2]/div/div[1]/main/article/section[1]/div/div/div/div[2]/div/div/div[{i}]'))
)
per_year_button.click()

WebDriverWait(driver, 30).until(
    EC.invisibility_of_element_located((By.XPATH, "//div[@class='pet-name' and text()='苦行骆驼']")))

time.sleep(10)
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "pet-card")))

page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')
pet_cards = soup.find_all(class_="pet-card")

for pet_card in pet_cards[len(pet_cards)-34:]:
    number = pet_card.find('div', class_='pet-title huiji-tt').get('data-name')
    name = pet_card.find('div', class_='pet-name').text
    image_url = pet_card.find('img', alt=True)
    high_res_url = image_url['srcset'].split(' ')[0]
    urllib.request.urlretrieve(high_res_url, f'images_{year}/No.{number}_{name}.png')
    print(f"download: No.{number}_{name}.png")


The chromedriver version (130.0.6723.58) detected in PATH at /usr/local/bin/chromedriver might not be compatible with the detected chrome version (131.0.6778.109); currently, chromedriver 131.0.6778.87 is recommended for chrome 131.*, so it is advised to delete the driver in PATH and retry


download: No.1220_圣藤草王.png
download: No.1219_金藤狂战.png
download: No.1218_金藤勇士.png
download: No.1217_金藤宝宝.png
download: No.1215_紫蔓守护.png
download: No.1214_紫蔓祭祀.png
download: No.1213_紫蔓宝宝.png
download: No.1252_水晶巨人.png
download: No.1251_水晶守卫.png
download: No.1250_晶灵.png
download: No.68_卡布托.png
download: No.1246_新年飞比.png
download: No.1245_万寿芝灵.png
download: No.1244_宝芝灵.png
download: No.1243_小芝灵.png
download: No.1242_转世灵菇.png
download: No.1241_幽灵菇.png
download: No.1240_小灵菇.png
download: No.1229_ 耳朵.png
download: No.1228_长耳兔.png
download: No.1227_红小兔.png
download: No.1226_小白.png
download: No.1225_小小白.png
download: No.1224_白宝宝.png
download: No.1239_千年雪莲.png
download: No.1238_莲依依.png
download: No.1235_火麒麟.png
download: No.1234_麒麟.png
download: No.1233_火精灵.png
download: No.1232_利爪黑熊.png
download: No.1231_黑熊.png
download: No.1230_小黑熊.png
download: No.1237_不死之参.png
download: No.1236_参果果.png


Seer

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import ElementNotInteractableException, NoSuchElementException
from bs4 import BeautifulSoup

import requests
import pandas as pd
import urllib.request
import time

In [ ]:
# Terminal: /Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome --remote-debugging-port=9222
def create_driver():
    chrome_options = Options()
    chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
    driver = webdriver.Chrome(options=chrome_options)
    return driver


### 4399

In [ ]:
url = 'https://news.4399.com/seer/jinglingdaquan/'
driver = create_driver()
driver.get(url)

mode_button=driver.find_element(By.XPATH,'//*[@id="tab4"]/li[2]/a')
mode_button.click()

while True:
    try:
        more_button=driver.find_element(By.XPATH,'/html/body/div[6]/div[5]/a')
        more_button.click()
        time.sleep(1)
    except ElementNotInteractableException:
        break

In [ ]:
pet_cards=driver.find_elements(By.CLASS_NAME,'jl_list2')
pet_cards = pet_cards[0].text
pet_cards = pet_cards.split('\n')
pet_cards = [x for x in pet_cards if not ('形态等级'in x or '种族值'in x or '专属技能'in x or '查询捕捉'in x or '详细信息' in x)]
print(pet_cards)

['米格罗斯', '精灵ID：4700', '精灵属性：地面机械', '霜爪雪貂', '精灵ID：4701', '精灵属性：冰暗影', '绮影罗心·织焰', '精灵ID：4696', '精灵属性：火神秘', '华光之翼·杰洛恩', '精灵ID：4697', '精灵属性：光飞行', '希尔琳', '精灵ID：4698', '精灵属性：自然神秘', '伊塔', '精灵ID：4699', '精灵属性：机械', '西塔·能源型', '精灵ID：4695', '精灵属性：机械火', '星光·蘑菇怪', '精灵ID：4693', '精灵属性：草', '石晶晶', '精灵ID：4694', '精灵属性：远古地面', '变异鳄兰', '精灵ID：4690', '精灵属性：草暗影', '墨韵', '精灵ID：4691', '精灵属性：水龙', '无间巨兽', '精灵ID：4692', '精灵属性：超能', '库洛妮', '精灵ID：4688', '精灵属性：冰神秘', '艾瑞吉', '精灵ID：4689', '精灵属性：远古草', '帕诺·绿光武装', '精灵ID：4686', '精灵属性：草机械', '小妮', '精灵ID：4687', '精灵属性：冰神秘', '帕诺·烈焰武装', '精灵ID：4684', '精灵属性：机械火', '云盈', '精灵ID：4685', '精灵属性：混沌飞行', '迪迦奥特曼', '精灵ID：4678', '精灵属性：光超能', '赛罗奥特曼', '精灵ID：4679', '精灵属性：圣灵战斗', '贝利亚奥特曼', '精灵ID：4680', '精灵属性：远古邪灵', '帕诺·海啸武装', '精灵ID：4683', '精灵属性：水机械', '空洄星徒', '精灵ID：4682', '精灵属性：光神秘', '神觉·米斯蒂克', '精灵ID：4676', '精灵属性：神秘', '星辰万象·天启星魂', '精灵ID：4677', '精灵属性：轮回神秘', '伏奥诺', '精灵ID：4681', '精灵属性：电', '无极使者', '精灵ID：4674', '精灵属性：次元', '招财', '精灵ID：4675', '精灵属性：地暗影', '噬夜急袭·库拉塔克', '精灵ID：4671', '精灵属性：暗影火', '月光·阿尔莫尼', '精灵ID：4672'

In [ ]:
names = []
numbers = []
elements = []

for index, info in enumerate(pet_cards):
    if index % 3 == 0 and info: 
        names.append(info)
    elif index % 3 == 1 and '精灵ID' in info:
        number = info.split('：')[-1]
        numbers.append(number)
    elif index % 3 == 2 and '精灵属性' in info:
        element = info.split('：')[-1]
        elements.append(element)

print(names)
print(numbers)
print(elements)

['米格罗斯', '霜爪雪貂', '绮影罗心·织焰', '华光之翼·杰洛恩', '希尔琳', '伊塔', '西塔·能源型', '星光·蘑菇怪', '石晶晶', '变异鳄兰', '墨韵', '无间巨兽', '库洛妮', '艾瑞吉', '帕诺·绿光武装', '小妮', '帕诺·烈焰武装', '云盈', '迪迦奥特曼', '赛罗奥特曼', '贝利亚奥特曼', '帕诺·海啸武装', '空洄星徒', '神觉·米斯蒂克', '星辰万象·天启星魂', '伏奥诺', '无极使者', '招财', '噬夜急袭·库拉塔克', '月光·阿尔莫尼', '月影·阿德洛卡', '紫烬魔兽', '月光飞贼', '龙灵战士', '尚克斯', '重塑之子·利塞特', '拟态绞索', '雷瀑飞雪·桑多姆', '凌虎·关银屏', '菠萝霸王', '无为觉者', '仿生兽·库比', '芙拉尔', '飘香小锅', '浮掠双刃·卓尔布', '锚点相座', '安蒂娅', '娜娜薇尔', '鲸沫', '天阶机器人', '库恩琪', '风暴龙灵', '时钟试炼官·极', '星光·丽莎布布', '星光·鲁斯王', '星光·魔焰猩猩', '克雷弗德', '米特拉异虫', '逆世之瞳·哀目黑子', '蓓丽安特', '佩因尔', '枫落殇·西叶', '西塔伦', '绒绒菊', '阳光菊', '蜜桃鸟', '黄桃鸟', '腐坏幽魔·克劳里菲', '佩里德', '卡威治', '塞弥丝珂', '霜之荆棘·蒙鲁特', '洛克斯', '圣血猎者·赫布洛', '伏尔格安', '夏厄普', '赫斯提亚', '裂岩·艾克特希', '阿兜', '雾灯徘徊者', '希拓', '佩洛瑟斯', '贝利弗', '孤烛灯使', '猛虎之刃·泰伯', '艾克特希', '罗萨', '罗萨尔', '佩斯莉', '魂弦', '渊枭', '尘息巨像', '克罗恩', '小卷', '无限·欧根尼克', '加尔鲁特', '小灯龙', '贾斯', '萨芙凯特', '灵巢之主·索杰德尔', '斯布林蒂', '露琪', '金瑞福龙', '织灵者·维伊', '卡米拉', '摆渡者·费雷', '阿帕', '索恩', '夜居魔使', '血刎剑姬', '古拉库', '曜焰万象·文森特', '覆骸冥土', '蛇舞·天鸣', '茵茵', '空元行者', '斯诺椰', '黑灵灵', '

In [ ]:
link_urls = []
i=1
while True:
    try:
        card_link=driver.find_element(By.XPATH,f'/html/body/div[6]/ul[2]/li[{i}]/a')
        link_url = card_link.get_attribute('href')
        link_urls.append(link_url)
        i+=1
    except NoSuchElementException:
        break
    

In [ ]:
df = pd.DataFrame({'No.': [int(number) for number in numbers],
                   'Name': names,
                   'Elements': elements,
                   'URL':link_urls})
df_sorted = df.sort_values(by='No.', ascending=True)
print(df_sorted)
csv_file_path = 'seer_data_4399.txt'
df_sorted.to_csv(csv_file_path, index=False,sep='\t', encoding='utf-8') 

       No.  Name Elements                                                URL
3165     3   布布花        草         http://news.4399.com/seer/tujian/47237.htm
3164     6   巴鲁斯        水         http://news.4399.com/seer/tujian/47223.htm
3163     9  烈焰猩猩        火         http://news.4399.com/seer/tujian/47227.htm
3167    12   波克尔       飞行         http://news.4399.com/seer/tujian/47228.htm
3166    15   天雷鼠        电         http://news.4399.com/seer/tujian/47233.htm
...    ...   ...      ...                                                ...
4     4698   希尔琳     自然神秘  http://news.4399.com/gonglue/seer/tujian/99819...
5     4699    伊塔       机械  http://news.4399.com/gonglue/seer/tujian/99819...
0     4700  米格罗斯     地面机械  http://news.4399.com/gonglue/seer/tujian/99838...
1     4701  霜爪雪貂      冰暗影  http://news.4399.com/gonglue/seer/tujian/99838...
1233  5000  圣灵谱尼       神灵  http://news.4399.com/gonglue/seer/tujian/77871...

[3282 rows x 4 columns]


### 4399 detail

In [ ]:
url = 'https://news.4399.com/seer/jinglingdaquan/'

# Terminal: /Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome --remote-debugging-port=9222
def create_driver():
    chrome_options = Options()
    chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

driver = create_driver()
driver.get(url)

The chromedriver version (130.0.6723.58) detected in PATH at /usr/local/bin/chromedriver might not be compatible with the detected chrome version (131.0.6778.109); currently, chromedriver 131.0.6778.87 is recommended for chrome 131.*, so it is advised to delete the driver in PATH and retry


In [ ]:
def process_link(driver, index):
    li_xpath = f'//*[@id="state"]/div[1]/ul/li[{index}]'
    time.sleep(1)
    li_element = driver.find_element(By.XPATH, li_xpath)
    span_element = None
    try:
        span_element = li_element.find_element(By.XPATH, './/span')
    except NoSuchElementException:
        return None, None, None, None
    
    img_xpath = f'//*[@id="state"]/div[1]/ul/li[{index}]/a/img'
    time.sleep(1)
    img = driver.find_element(By.XPATH, img_xpath)
    pic = img.get_attribute('src')
    img.click()
    number = driver.find_element(By.XPATH, f'//*[@id="state"]/div[2]/dl[{index}]/dd/span[1]').text.replace('精灵序号：', '')
    name = driver.find_element(By.XPATH, f'//*[@id="state"]/div[2]/dl[{index}]/dt').text
    element = driver.find_element(By.XPATH, f'//*[@id="state"]/div[3]/dl[{index}]/dt/i').text
    return name, number, element, pic

    

In [ ]:
names = []
numbers = []
elements = []
pics = []

data=pd.read_csv('seer_data_4399.txt', sep='\t')
link_urls = data['URL']
link_urls = link_urls[665:]
for link_url in link_urls:
    driver.get(link_url)
    
    for index in range(1,4):
        name, number, element, pic = process_link(driver, index)
        if name and number and element: 
            names.append(name)
            numbers.append(number)
            elements.append(element)
            pics.append(pic)
            print(f'{number}, {name}, {element}, {pic}')
    
    time.sleep(1)

df = pd.DataFrame({'No.': [int(number) for number in numbers],
                   'Name': names,
                   'Elements': elements,
                   'image_url': pics})

df_sorted = df.sort_values(by='No.', ascending=True)
print(df_sorted)
csv_file_path = 'seer_data_4399_1.txt'
df_sorted.to_csv(csv_file_path, index=False,sep='\t', encoding='utf-8') 


In [ ]:
df=pd.read_csv('seer_data_4399_1.txt',sep='\t')
df.head()


,No.,Name,Elements,image_url
0,1,布布种子,草系,https://newsimg.5054399.com/uploads/userup/150...
1,2,布布草,草系,https://newsimg.5054399.com/uploads/userup/150...
2,3,布布花,草系,https://newsimg.5054399.com/uploads/userup/150...
3,4,伊优,水系,https://newsimg.5054399.com/uploads/userup/151...
4,5,尤里安,水系,https://newsimg.5054399.com/uploads/userup/151...


### wiki

In [ ]:
url = 'https://seerelvesinpokemonstyle.fandom.com/zh/wiki/%E7%B2%BE%E7%81%B5%E5%88%97%E8%A1%A8%EF%BC%88%E6%8C%89%E5%85%A8%E5%9F%9F%E5%9B%BE%E9%89%B4%E7%BC%96%E5%8F%B7%EF%BC%89?variant=zh-cn'
driver = create_driver()
driver.get(url)
time.sleep(3)
table=driver.find_element(By.XPATH,'/html/body/div[6]/div[4]/div[4]/main/div[3]/div/div/table[1]')

names = []
numbers = []
elements = []
for i in range(1,192):
    number=driver.find_element(By.XPATH,f'/html/body/div[6]/div[4]/div[4]/main/div[3]/div/div/table[1]/tbody/tr[{i}]/td[1]').text
    
    try:
        name=driver.find_element(By.XPATH,f'/html/body/div[6]/div[4]/div[4]/main/div[3]/div/div/table[1]/tbody/tr[{i}]/td[3]/a').text
    except NoSuchElementException:
        try:
            name=driver.find_element(By.XPATH,f'//*[@id="mw-content-text"]/div/table[1]/tbody/tr[{i}]/td[3]/span').text
        except NoSuchElementException:
            name = None

    ele = driver.find_element(By.XPATH, f'/html/body/div[6]/div[4]/div[4]/main/div[3]/div/div/table[1]/tbody/tr[{i}]/td[4]').text
    ele2 = None  
    try:
        ele2 = driver.find_element(By.XPATH, f'/html/body/div[6]/div[4]/div[4]/main/div[3]/div/div/table[1]/tbody/tr[{i}]/td[5]').text
    except NoSuchElementException:
        pass 

    if ele2:  
        ele = [ele, ele2]
        ele = '，'.join(ele)  
    else:
        ele = ele
    
    # fig = driver.find_element(By.XPATH,f'/html/body/div[6]/div[4]/div[4]/main/div[3]/div/div/table[1]/tbody/tr[{i}]/td[2]/a')
    # fig_url=fig.get_attribute('href')
    # urllib.request.urlretrieve(fig_url, f'images/No.{number}_{name}.png')
    # print(f'downlaod: No.{number}_{name}.png')
    
    names.append(name)
    numbers.append(number)
    elements.append(ele)

df = pd.DataFrame({'No.': [int(number) for number in numbers],
                   'Name': names,
                   'Elements': elements})

csv_file_path = 'seer_data_wiki.txt'
df.to_csv(csv_file_path, index=False,sep='\t', encoding='utf-8') 

    



The chromedriver version (130.0.6723.58) detected in PATH at /usr/local/bin/chromedriver might not be compatible with the detected chrome version (131.0.6778.109); currently, chromedriver 131.0.6778.87 is recommended for chrome 131.*, so it is advised to delete the driver in PATH and retry


change chinese to english

In [ ]:
import os
from pypinyin import lazy_pinyin
import pandas as pd

In [ ]:
directory_path = '/Users/fonglengsut/Desktop/introduction of ds/group project/other games/seer/images'
rename_files(directory_path)

In [ ]:
def add_english_names_to_txt(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)
            df=pd.read_csv(file_path,sep='\t')
            if 'Name' in df.columns:
                df['Eng_Name'] = df['Name'].apply(lambda x: ''.join(lazy_pinyin(x)))
                df.to_csv(file_path, index=False, sep=',')

In [ ]:
directory_path = '/Users/fonglengsut/Desktop/introduction of ds/group project/other games/seer'
add_english_names_to_txt(directory_path)

Get pokemon data from pokeAPI

In [ ]:
import requests
import json
import os

# 定义保存图片的目录
save_directory = "data/pokemon_images"
os.makedirs(save_directory, exist_ok=True)

# 获取所有宝可梦的基本信息
url = "https://pokeapi.co/api/v2/pokemon?limit=10000"
response = requests.get(url)
data = response.json()

# 创建一个字典来存储宝可梦的名称和属性
pokemon_info = {}

# 遍历每个宝可梦
for pokemon in data['results']:
    pokemon_name = pokemon['name']
    pokemon_url = pokemon['url']
    response = requests.get(pokemon_url)
    pokemon_data = response.json()

    # 获取宝可梦的属性
    types = pokemon_data['types']
    type_names = [type_info['type']['name'] for type_info in types]

    # 获取宝可梦的图片链接
    pic1_url = pokemon_data['sprites']['other']['official-artwork']['front_default']
    pic2_url = pokemon_data['sprites']['other']['home']['front_default']
    pic3_url = pokemon_data['sprites']['front_default']

    # 下载并保存图片
    for i, pic_url in enumerate([pic1_url, pic2_url, pic3_url], start=1):
        if pic_url:
            img_response = requests.get(pic_url)
            if img_response.status_code == 200:
                img_name = f"{pokemon_name}_pic{i}.png"
                img_path = os.path.join(save_directory, img_name)
                with open(img_path, 'wb') as file:
                    file.write(img_response.content)

    # 将宝可梦的名称和属性存储到字典中
    pokemon_info[pokemon_name] = {
        "types": type_names,
        "images": [os.path.join(save_directory, f"{pokemon_name}_pic{i}.png") for i in range(1, 4) if eval(f"pic{i}_url")]
    }

# 将字典转换为 JSON 格式并保存到文件中
with open('data/pokemon_info.json', 'w') as json_file:
    json.dump(pokemon_info, json_file, indent=4)

print(f"所有宝可梦的名称和属性已保存到 pokemon_info.json 文件中，图片保存在 {save_directory} 目录中。")

所有宝可梦的名称和属性已保存到 pokemon_info.json 文件中，图片保存在 data/pokemon_images 目录中。


In [ ]:
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image

# 定义自定义数据集类
class PokemonDataset(Dataset):
    def __init__(self, data_dict, transform=None):
        self.data_dict = data_dict
        self.transform = transform
        self.image_files = []
        self.labels = []
        
        # 遍历数据字典，获取所有图像文件路径和对应的标签（属性）
        for pokemon_name, info in data_dict.items():
            for img_path in info['images']:
                self.image_files.append(img_path)
                self.labels.append(info['types'])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert("RGB")
        
        # 获取宝可梦的标签（属性）
        labels = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        # 将字符串标签转换为整数标签
        label_map = {'grass': 0, 'poison': 1, 'fire': 2, 'water': 3, 'electric': 4, 'ice': 5, 'fighting': 6,
                     'ground': 7, 'flying': 8, 'psychic': 9, 'bug': 10, 'rock': 11, 'ghost': 12, 'dark': 13,
                     'dragon': 14, 'steel': 15, 'fairy': 16, 'normal': 17}
        labels = [label_map[label] for label in labels]
        
        # 将标签转换为多标签二进制格式
        multi_label = torch.zeros(len(label_map), dtype=torch.float32)
        for label in labels:
            multi_label[label] = 1.0
        
        return image, multi_label

# 读取数据字典
with open('data/pokemon_info.json', 'r') as f:
    data_dict = json.load(f)

# 定义训练数据的转换
data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

# 创建数据集
pokemon_dataset = PokemonDataset(data_dict=data_dict, transform=data_transforms)

# 将数据集分割为训练集、验证集和测试集（80%、10%、10%）
train_size = int(0.8 * len(pokemon_dataset))
val_size = int(0.1 * len(pokemon_dataset))
test_size = len(pokemon_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(pokemon_dataset, [train_size, val_size, test_size])

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)


In [ ]:
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader, random_split
import cv2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 定义自定义数据集类
class PokemonDataset(Dataset):
    def __init__(self, data_dict, transform=None):
        self.data_dict = data_dict
        self.transform = transform
        self.image_files = []
        self.labels = []
        
        # 遍历数据字典，获取所有图像文件路径和对应的标签（属性）
        for pokemon_name, info in data_dict.items():
            for img_path in info['images']:
                self.image_files.append(img_path)
                self.labels.append(info['types'])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        
        # 使用OpenCV加载图像
        image = cv2.imread(img_path)
        if image is None:
            print(f"UnidentifiedImageError: cannot identify image file '{img_path}'")
            return self.__getitem__((idx + 1) % len(self.image_files))
        
        # 将图像从BGR转换为RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 获取宝可梦的标签（属性）
        labels = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        # 将字符串标签转换为整数标签
        label_map = {'grass': 0, 'poison': 1, 'fire': 2, 'water': 3, 'electric': 4, 'ice': 5, 'fighting': 6,
                     'ground': 7, 'flying': 8, 'psychic': 9, 'bug': 10, 'rock': 11, 'ghost': 12, 'dark': 13,
                     'dragon': 14, 'steel': 15, 'fairy': 16, 'normal': 17}
        labels = [label_map[label] for label in labels]
        
        # 将标签转换为多标签二进制格式
        multi_label = torch.zeros(len(label_map), dtype=torch.float32)
        for label in labels:
            multi_label[label] = 1.0
        
        return image, multi_label

# 定义函数来提取图像特征
def extract_features(model, dataloader):
    model.eval()
    features = []
    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            features.append(outputs.cpu().numpy())
    return np.concatenate(features)


# 读取数据字典
with open('data/pokemon_info.json', 'r') as f:
    data_dict = json.load(f)

# 定义训练数据的转换
data_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

# 创建数据集
pokemon_dataset = PokemonDataset(data_dict=data_dict, transform=data_transforms)

# 将数据集分割为训练集、验证集和测试集（80%、10%、10%）
train_size = int(0.8 * len(pokemon_dataset))
val_size = int(0.1 * len(pokemon_dataset))
test_size = len(pokemon_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(pokemon_dataset, [train_size, val_size, test_size])

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# 检查是否有可用的GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 加载预训练的ResNet18模型
model = models.resnet18(pretrained=True)

# 修改最后一层以匹配属性的数量（假设有18种可能的属性）
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 18)

# 将模型移动到GPU
model = model.to(device)

# 定义损失函数和优化器
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 训练循环
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs_labels in train_loader:
        if inputs_labels is None:
            continue
        
        inputs, labels = inputs_labels
        
        # 将输入和标签移动到GPU
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

    # 验证循环
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs_labels in val_loader:
            if inputs_labels is None:
                continue
            
            inputs, labels = inputs_labels
            
            # 将输入和标签移动到GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    
    print(f"Validation Loss: {val_loss/len(val_loader)}")

print("Training complete.")

# 测试循环（可选）
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs_labels in test_loader:
        if inputs_labels is None:
            continue
        
        inputs, labels = inputs_labels
        
        # 将输入和标签移动到GPU
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        test_loss += loss.item()

print(f"Test Loss: {test_loss/len(test_loader)}")



Using device: cuda:0
Epoch 1/10, Loss: 0.30217974563491157
Validation Loss: 0.28418717986529635
Epoch 2/10, Loss: 0.2739431640721973
Validation Loss: 0.2694471507650061
Epoch 3/10, Loss: 0.26146089968903696
Validation Loss: 0.2585177209573923
Epoch 4/10, Loss: 0.2513724207839509
Validation Loss: 0.24945543199470363
Epoch 5/10, Loss: 0.2423582907881916
Validation Loss: 0.24683019611024365
Epoch 6/10, Loss: 0.23483559973730941
Validation Loss: 0.23940254087300644
Epoch 7/10, Loss: 0.22669326286241798
Validation Loss: 0.2360087030941678
Epoch 8/10, Loss: 0.21843267285260204
Validation Loss: 0.22845934024176648
Epoch 9/10, Loss: 0.2098351487976745
Validation Loss: 0.22521687905813
Epoch 10/10, Loss: 0.20106783593218253
Validation Loss: 0.22107784250347884
Training complete.
Test Loss: 0.22897303227296809


In [ ]:
# 示例：查找与查询图像最相似的三个宝可梦图像
# 定义函数来找到最相似的宝可梦图像
def find_similar_images(query_image_path, model, feature_extractor, dataset, dataloader, top_k=3):
    # 加载查询图像并进行预处理
    query_image = cv2.imread(query_image_path)
    if query_image is None:
        raise ValueError(f"Cannot identify image file '{query_image_path}'")
    
    query_image = cv2.cvtColor(query_image, cv2.COLOR_BGR2RGB)
    query_image = data_transforms(query_image).unsqueeze(0).to(device)
    
    # 提取查询图像的特征
    with torch.no_grad():
        query_features = feature_extractor(query_image).cpu().numpy()
    
    # 提取数据集中所有图像的特征
    dataset_features = extract_features(feature_extractor, dataloader)
    
    # 计算余弦相似度
    similarities = cosine_similarity(query_features, dataset_features)
    
    # 找到最相似的图像索引
    top_k_indices = np.argsort(similarities[0])[::-1][:top_k]
    
    # 返回最相似的图像路径和相似度分数
    similar_images = [(dataset.image_files[idx], similarities[0][idx]) for idx in top_k_indices]
    
    return similar_images

query_image_path = 'data/palworld_images/Anubis.png'
similar_images = find_similar_images(query_image_path, model, model, pokemon_dataset, train_loader, top_k=3)
print("Most similar images:")
for img_path, score in similar_images:
    print(f"Image: {img_path}, Similarity Score: {score}")

Most similar images:
Image: data/pokemon_images\passimian_pic3.png, Similarity Score: 0.9777513146400452
Image: data/pokemon_images\incineroar_pic1.png, Similarity Score: 0.9698570966720581
Image: data/pokemon_images\azurill_pic1.png, Similarity Score: 0.969271719455719


In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

def find_similar_images_euclidean(query_image_path, model, feature_extractor, dataset, dataloader, top_k=3):
    query_image = cv2.imread(query_image_path)
    if query_image is None:
        raise ValueError(f"Cannot identify image file '{query_image_path}'")

    query_image = cv2.cvtColor(query_image, cv2.COLOR_BGR2RGB)
    query_image = data_transforms(query_image).unsqueeze(0).to(device)

    with torch.no_grad():
        query_features = feature_extractor(query_image).cpu().numpy()

    dataset_features = extract_features(feature_extractor, dataloader)

    distances = euclidean_distances(query_features, dataset_features)

    top_k_indices = np.argsort(distances[0])[:top_k]

    similar_images = [(dataset.image_files[idx], distances[0][idx]) for idx in top_k_indices]

    return similar_images

query_image_path = 'data/palworld_images/Anubis.png'
similar_images = find_similar_images_euclidean(query_image_path, model, model, pokemon_dataset, train_loader, top_k=3)
print("Most similar images:")
for img_path, score in similar_images:
    print(f"Image: {img_path}, Distance: {score}")

Most similar images:
Image: data/pokemon_images\cetoddle_pic3.png, Distance: 3.8211874961853027
Image: data/pokemon_images\bastiodon_pic1.png, Distance: 3.8765923976898193
Image: data/pokemon_images\wyrdeer_pic1.png, Distance: 4.010998249053955


In [ ]:
from torchvision.models import vgg16

# 加载预训练的VGG16模型
vgg_model = vgg16(pretrained=True)
vgg_model.classifier = nn.Sequential(*list(vgg_model.classifier.children())[:-1])  # 移除最后一层
vgg_model = vgg_model.to(device)

def extract_vgg_features(model, dataloader):
    model.eval()
    features = []
    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            features.append(outputs.cpu().numpy())
    return np.concatenate(features)

query_image_path = 'data/palworld_images/Anubis.png'
similar_images = find_similar_images(query_image_path, vgg_model, vgg_model, pokemon_dataset, train_loader, top_k=3)
print("Most similar images:")
for img_path, score in similar_images:
    print(f"Image: {img_path}, Similarity Score: {score}")

Most similar images:
Image: data/pokemon_images\cradily_pic3.png, Similarity Score: 0.514254093170166
Image: data/pokemon_images\absol_pic2.png, Similarity Score: 0.49518799781799316
Image: data/pokemon_images\drowzee_pic2.png, Similarity Score: 0.4846184253692627


In [ ]:
import cv2
import imagehash
import numpy as np
 
def find_similar_images_hash(query_image_path, dataset, top_k=3):
    # 使用 OpenCV 读取图像
    query_image = cv2.imread(query_image_path)
    # OpenCV 读取的图像是 BGR 格式，需要转换为 RGB 格式
    query_image_rgb = cv2.cvtColor(query_image, cv2.COLOR_BGR2RGB)
    # 将图像转换为 Pillow 图像对象以使用 imagehash 库
    query_image_pil = Image.fromarray(query_image_rgb)
    query_hash = imagehash.phash(query_image_pil)
 
    image_hashes = []
    for img_path in dataset.image_files:
        # 使用 OpenCV 读取图像
        img = cv2.imread(img_path)
        # 转换为 RGB 格式
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # 转换为 Pillow 图像对象
        img_pil = Image.fromarray(img_rgb)
        # 计算图像哈希
        img_hash = imagehash.phash(img_pil)
        image_hashes.append((img_path, img_hash))
 
    similarities = [(img_path, 1 - (query_hash - img_hash) / len(query_hash.hash)**2) for img_path, img_hash in image_hashes]
    similarities.sort(key=lambda x: x[1], reverse=True)
 
    return similarities[:top_k]
 
query_image_path = 'data/palworld_images/Anubis.png'
# 假设 pokemon_dataset 是一个具有 image_files 属性的对象，该属性包含图像文件路径的列表
# 注意：你需要确保 pokemon_dataset 已经被正确定义并包含有效的图像文件路径
similar_images = find_similar_images_hash(query_image_path, pokemon_dataset, top_k=3)
print("Most similar images:")
for img_path, score in similar_images:
    print(f"Image: {img_path}, Similarity Score: {score}")

Most similar images:
Image: data/pokemon_images\pichu_pic3.png, Similarity Score: 0.75
Image: data/pokemon_images\chinchou_pic3.png, Similarity Score: 0.71875
Image: data/pokemon_images\porygon2_pic2.png, Similarity Score: 0.71875
